In [1]:
# Working example
import pandas as pd
from cytominer_eval import evaluate

# Load Data
path = "profiles/NCP_PILOT_3/BR_NCP_PILOT_3/BR_NCP_PILOT_3_normalized_variable_selected.csv.gz"

df = pd.read_csv(path)

# Define important function arguments
meta_features = df.columns[df.columns.str.startswith("Metadata_")]
features = df.drop(meta_features, axis="columns").columns.tolist()


In [2]:
# Metadata_group is Metadata_line_condition + Metadata_plating_density
# Metadata_replicate is Metadata_compound_ID

df.head(5)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_line_condition,Metadata_compound_ID,Metadata_replicate,Metadata_group,...,Neurites_Texture_InfoMeas2_RNA_20_03,Neurites_Texture_InverseDifferenceMoment_AGP_10_02,Neurites_Texture_InverseDifferenceMoment_AGP_20_02,Neurites_Texture_InverseDifferenceMoment_DNA_5_00,Neurites_Texture_SumAverage_AGP_10_00,Neurites_Texture_SumAverage_RNA_20_00,Neurites_Texture_SumVariance_Brightfield_10_03,Neurites_Texture_SumVariance_DNA_20_03,Neurites_Texture_SumVariance_ER_20_03,Neurites_Texture_Variance_AGP_5_02
0,BR_NCP_PILOT_3,A01,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A01,2500,control,Untreated,Untreated,control_2500,...,0.68736,0.32353,0.38678,-1.333400,0.84262,0.43418,2.85160,-0.065548,1.05590,1.8216
1,BR_NCP_PILOT_3,A02,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A02,10000,control,Untreated,Untreated,control_10000,...,1.42290,-1.49340,-1.79440,-0.053887,1.81270,2.02500,4.02830,-0.377890,1.38140,2.0071
2,BR_NCP_PILOT_3,A03,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A03,2500,control,BRD-K79131256-001-17-9,BRD-K79131256-001-17-9,control_2500,...,0.83461,-0.84041,-0.30974,0.854620,1.29650,0.03451,0.90154,-0.094023,0.67013,2.7642
3,BR_NCP_PILOT_3,A04,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A04,10000,control,BRD-K36207157-001-09-6,BRD-K36207157-001-09-6,control_10000,...,1.66050,-1.73520,-1.53300,-0.472280,1.98300,1.34460,1.43450,0.477740,1.00020,2.0469
4,BR_NCP_PILOT_3,A05,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A05,2500,control,Untreated,Untreated,control_2500,...,2.11990,1.42280,1.51170,0.554710,-0.27509,-0.17327,0.92286,-0.267170,0.50922,1.2480


In [3]:
df.sample(5)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_line_condition,Metadata_compound_ID,Metadata_replicate,Metadata_group,...,Neurites_Texture_InfoMeas2_RNA_20_03,Neurites_Texture_InverseDifferenceMoment_AGP_10_02,Neurites_Texture_InverseDifferenceMoment_AGP_20_02,Neurites_Texture_InverseDifferenceMoment_DNA_5_00,Neurites_Texture_SumAverage_AGP_10_00,Neurites_Texture_SumAverage_RNA_20_00,Neurites_Texture_SumVariance_Brightfield_10_03,Neurites_Texture_SumVariance_DNA_20_03,Neurites_Texture_SumVariance_ER_20_03,Neurites_Texture_Variance_AGP_5_02
350,BR_NCP_PILOT_3,O15,BR_NCP_PILOT_3,BR_NCP_PILOT_3,O15,2500,deletion,Untreated,Untreated,deletion_2500,...,-0.74035,1.06120,1.50770,-0.46551,-0.907420,-0.695280,-0.57539,0.37736,-0.58333,-0.63572
298,BR_NCP_PILOT_3,M11,BR_NCP_PILOT_3,BR_NCP_PILOT_3,M11,2500,control,BRD-K21680192-300-14-4,BRD-K21680192-300-14-4,control_2500,...,-1.96600,-2.39840,-2.55160,0.43260,0.864300,-1.611100,-0.68535,-1.23480,-1.09080,0.11174
171,BR_NCP_PILOT_3,H04,BR_NCP_PILOT_3,BR_NCP_PILOT_3,H04,7500,deletion,Untreated,Untreated,deletion_7500,...,-0.59571,-0.46925,-0.65151,-0.74758,-0.674290,0.018072,-0.21434,0.73135,-0.72519,-0.92319
193,BR_NCP_PILOT_3,I02,BR_NCP_PILOT_3,BR_NCP_PILOT_3,I02,10000,control,Untreated,Untreated,control_10000,...,0.78340,-0.40464,-0.30452,-0.27664,0.052507,0.538530,2.32360,4.01420,0.43459,0.61312
378,BR_NCP_PILOT_3,P19,BR_NCP_PILOT_3,BR_NCP_PILOT_3,P19,5000,deletion,Untreated,Untreated,deletion_5000,...,-0.27202,-1.04740,-1.14320,-1.65790,0.815030,2.188100,1.37290,1.40690,-0.13012,0.52995


In [4]:
# Evaluate profile quality using grit

control_perts = [
    "Untreated"
]

replicate_groups = {
    "replicate_id": "Metadata_replicate",
    "group_id": "Metadata_Plate",
}

grit_results_df = evaluate(
    profiles=df,
    features=features,
    meta_features=meta_features,
    replicate_groups=replicate_groups,
    operation="grit",
    grit_control_perts=control_perts,
)

grit_results_df

,perturbation,group,grit
0,BRD-K15108141-003-04-7,BR_NCP_PILOT_3,0.039541
1,BRD-K21680192-300-14-4,BR_NCP_PILOT_3,0.029373
2,BRD-K36207157-001-09-6,BR_NCP_PILOT_3,0.058899
3,BRD-K79131256-001-17-9,BR_NCP_PILOT_3,0.019049
4,Untreated,BR_NCP_PILOT_3,-0.058353


In [5]:
# But the grouping we actually want is by group_id (which is `Metadata_line_condition_Metadata_plating_density`)

replicate_groups = {
    "replicate_id": "Metadata_replicate",
    "group_id": "Metadata_plating_density",
}

grit_results_df = evaluate(
    profiles=df,
    features=features,
    meta_features=meta_features,
    replicate_groups=replicate_groups,
    operation="grit",
    grit_control_perts=control_perts,
)

grit_results_df

AssertionError: grit is calculated for each perturbation independently